# translate

In [23]:
from googletrans import Translator
import langid
import pandas as pd

In [24]:
def detect_Translator(text):
    translator = Translator()
    try:
        detection = translator.detect(text)
        return detection
    except Exception as e:
        return "error:- "+e

In [25]:
def detect_langid(text):
    try:
        lang, confidence = langid.classify("Sample Hinglish text here")
        return lang
    except Exception as e:
        return "error:- "+e

In [29]:
def test(df : pd.DataFrame):
    for index,row in df.iterrows():
        langid = detect_langid(row['Comment'])
        Translator = detect_Translator(row['Comment'])
        print(f"{row['Comment']}, langid :- {langid}, Translator :- {Translator}")
        print('\n')

In [ ]:
df=pd.read_csv(r'C:\Users\tarun\Desktop\Youtube-Comments-Sentiments-Analysis\comments_df.csv')
test(df)

In [47]:
def detect_and_translate(text):
    translator = Translator()
    try:
        detection = translator.detect(text)
        print(detection)
        if detection.lang != 'en':
            translation = translator.translate(text, dest='en')
            if translation and translation.text:
                print(translation.text)
                print(translation)
                return translation.text
            else:
                return 'This is a neutral text'
        else:
            return text
    except Exception as e:
        print(f"Translation error: {e}")
        return 'This is a neutral text'

In [65]:
ans=detect_and_translate(text)
ans

Detected(lang=hi, confidence=None)
A desert will not die now, Betiichod
Translated(src=hi, dest=en, text=A desert will not die now, Betiichod, pronunciation=None, extra_data="{'confiden...")


'A desert will not die now, Betiichod'